In [73]:
import pandas as pd
!pip install xlrd

data=pd.read_csv('/content/all_restaurants_data.csv')
data.head()
#data.shape[0]
file_path = "/content/top_k_custom_score (2).xls"
top_k_custom_score = pd.read_csv(file_path, encoding='latin1')
print(top_k_custom_score.head())
top_k_custom_score.shape[0]

      restaurantName                      address  \
0       Piccolo Lord  corso San Maurizio 69 bis/g   
1        AltrimÃ©nti         via Monte Bianco 2/a   
2       Kirchsteiger        via prevosto Wieser 5   
3               Mima             via Madonnelle 9   
4  Trattoria 'petito             via Corridoni 14   

                                         description  \
0  Professional service in a welcoming, modern re...   
1  An informal and contemporary restaurant which ...   
2  This restaurant has long been one of the most ...   
3  Youâll be won over by the seasonal Mediterra...   
4  Located on the outskirts of ForlÃ¬âs charmin...   

                                             website  \
0              https://www.ristorantepiccololord.it/   
1                             https://altrimenti.eu/   
2  https://www.kirchsteiger.com/it/gastronomia/ri...   
3                   http://www.domo20.com/restaurant   
4                    https://www.trattoriapetito.it/   

       

1000

In [86]:
#!!!don't run this code!!! import df dataset in the following step
!pip install opencage
from opencage.geocoder import OpenCageGeocode
import pandas as pd
# Insert my API key of OpenCage
key='4101055b9fa84cbdbe8b71af558c30ec'
geocoder=OpenCageGeocode(key)
#obtain a list of unique cities
list_cities=data['city'].unique()
coordinates=[]
for i in list_cities:
    location=geocoder.geocode(i)
    if location:
        coordinates.append({"city": i, "latitude": location[0]['geometry']['lat'] , "longitude": location[0]['geometry']['lng']})
df=pd.DataFrame(coordinates)
df

,city,latitude,longitude
0,Gragnano,40.690000,14.521065
1,Popoli,42.171311,13.832817
2,Alba,46.015936,23.546850
3,Sorrento,40.624906,14.374836
4,Matera,40.447642,16.473574
...,...,...,...
1163,Capoterra,39.175313,8.971950
1164,Pisa,43.471472,10.679791
1165,Cardano,46.493717,11.393647
1166,Badalucco,43.915557,7.847295


In [80]:
#save the dataset 'df' because OpenCage allows only 2500 requests per day for free accounts
df.to_csv("coordinates.csv",index=False)


In [84]:
#for those running the code execute this cell to import 'df'
df=pd.read_csv('/content/coordinates.csv')
df.shape[0]

1168

In [88]:
import folium
import pandas as pd
#qui flavio in top_k_custom_score ci devi mettere il richiamo della tua funzione custom score
top_k_custom_score=top_k_custom_score.head(5)
#I've done an inner join between top_k_custom_score and data to add city information to the top_k_custom_score dataset
# I chose address as keys because restaurantName sometimes contains invalid values and the addition of city fails
top_k_custom_score_city=pd.merge(
    top_k_custom_score,
    data[['address','city']],
    on="address",
    how="inner"
)
#I've done an left join on the 'city' key between 'top_k_custom_score_city' and 'df' to add coordinates information to top_k_custom_score_city dataset
union=pd.merge(top_k_custom_score_city,df,on="city",how="left")
#create map centered to italy
map=folium.Map(location=[41.8719, 12.5674],zoom_start=6)
#determine the color of marker based on price range
def price_color(price):
    if price=="Economic":
        return "green"
    elif price=="Affordable":
        return "blue"
    elif price=="Expensive":
        return "orange"
    elif price=="Luxury":
        return "red"
#for each restaurant add the marker on the map
for i in range(len(union)):
    restaurant=union["restaurantName"].iloc[i]
    city=union["city"].iloc[i]
    address=union["address"].iloc[i]
    price_range=union["Price Range"].iloc[i]
    custom_score=union["Custom Score"].iloc[i]
    latitude=union["latitude"].iloc[i]
    longitude=union["longitude"].iloc[i]
#creation of popup with information on the map
    popup_text=f"<b>{restaurant}</b><br>{address},{city}<br>Price Range:{price_range}<br>Custom score:{custom_score:.4f}"
    folium.Marker(
        location=[latitude,longitude],
        popup=folium.Popup(popup_text,max_width=250),
        icon=folium.Icon(color=price_color(price_range))
    ).add_to(map)

#create legend with price range
legend= '''
<div style="
    position:fixed;
    bottom:10px;
    left:10px;
    width:150px;
    border:1px solid grey;
    background-color:white;
    padding:10px;
    z-index:9999;
">
    <b>Price Range Legend</b><br>
    <span style="color:green;">●</span> € Economic<br>
    <span style="color:blue;">●</span> €€ Affordable<br>
    <span style="color:orange;">●</span> €€€ Expensive<br>
    <span style="color:red;">●</span> €€€€ Luxury
</div>
'''

#add legend to map
map.get_root().html.add_child(folium.Element(legend))
#save map in an html file
map.save("top_k_restaurants_map.html")
